In [1]:
import keras
import tensorflow
import h5py
import numpy as np
import chowdhury_s_model_builder
import os

import tensorflow as tf
from keras.api.models import Sequential, Model, load_model
from keras.api.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization, LSTM, Input, GlobalAveragePooling2D
from keras.api.optimizers import Adam
from keras import regularizers

import tensorflow as tf
print(tf.__version__)

from sklearn.metrics import classification_report, multilabel_confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

2025-04-26 11:46:35.034496: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-26 11:46:35.253593: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745639195.335074     783 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745639195.355936     783 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745639195.526341     783 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

2.19.0


In [2]:
# Open the HDF5 file in read mode
with h5py.File("Data/Mid-Level_Perceptual_Features_Data.h5", "r") as hf:
    mid_level_train_data = hf["train"][:]  # Load the train dataset
    mid_level_label_data = hf["label"][:]  # Load the label dataset

print(mid_level_train_data.shape, mid_level_train_data.dtype)  # Check the shape and type
print(mid_level_label_data.shape, mid_level_label_data.dtype)

(5000, 256, 1292) float32
(5000, 7) int8


In [3]:
mid_level_train_data = mid_level_train_data[..., np.newaxis]

print(mid_level_train_data.shape, mid_level_train_data.dtype)  # Check the shape and type
print(mid_level_label_data.shape, mid_level_label_data.dtype)

(5000, 256, 1292, 1) float32
(5000, 7) int8


In [4]:
mid_level_input_shape = mid_level_train_data.shape[1], mid_level_train_data.shape[2], mid_level_train_data.shape[3]
print(mid_level_input_shape)

(256, 1292, 1)


In [5]:
model = load_model("fine-tuned_model.keras")
model.summary()

I0000 00:00:1745639216.748275     783 gpu_process_state.cc:208] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1745639216.751641     783 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 256, 1292, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 126, 644, 64)   │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 126, 644, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 126, 644, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 126, 644, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 322, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 63, 322, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 63, 322, 128)   │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 63, 322, 128)   │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 63, 322, 128)   │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 63, 322, 128)   │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 31, 161, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 31, 161, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 31, 161, 256)   │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 31, 161, 256)   │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 31, 161, 256)   │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 31, 161, 256)   │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 31, 161, 384)   │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 31, 161, 384)   │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 31, 161, 512)   │     1,769,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 31, 161, 512)   │         2,048 │
│ (BatchNormalization)            │                        │             

 Total params: 5,191,255 (19.80 MB)

 Trainable params: 67,591 (264.03 KB)

 Non-trainable params: 4,988,480 (19.03 MB)

 Optimizer params: 135,184 (528.07 KB)

In [14]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
x_train_split, x_val, y_train_split, y_val = train_test_split(
    mid_level_train_data, mid_level_label_data, test_size=0.5, random_state=42
)

In [12]:
model.compile(optimizer=Adam(learning_rate=3.1250e-05),
                           loss='binary_crossentropy', metrics=["accuracy"]
                           )

In [13]:
results = model.evaluate(x_val, y_val, batch_size=8, verbose=1)
print("Validation Loss:", results[0])
print("Validation Accuracy:", results[1])
# Predict the labels for the validation set

125/125 ━━━━━━━━━━━━━━━━━━━━ 19s 129ms/step - accuracy: 0.7782 - loss: 0.0018
Validation Loss: 0.0016471187118440866
Validation Accuracy: 0.7839999794960022


In [15]:
# Assuming 'model' is your trained model
y_pred = model.predict(x_val)

# Convert probabilities to binary labels (e.g., threshold at 0.5)
y_pred_binary = (y_pred > 0.5).astype(int)

2025-04-26 11:55:37.545836: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_275', 8 bytes spill stores, 8 bytes spill loads

2025-04-26 11:55:41.257379: E external/local_xla/xla/service/slow_operation_alarm.cc:73] Trying algorithm eng33{k2=2,k6=0,k13=2,k14=0,k22=2} for conv %cudnn-conv-bias-activation.27 = (f32[32,64,126,644]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,1,256,1292]{3,2,1,0} %bitcast.767, f32[64,1,5,5]{3,2,1,0} %bitcast.774, f32[64]{0} %bitcast.776), window={size=5x5 stride=2x2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="functional_1/conv2d_1/convolution" source_file="/home/nigelchua0412/.local/lib/python3.12/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_s

78/79 ━━━━━━━━━━━━━━━━━━━━ 0s 806ms/step

2025-04-26 11:59:14.226030: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_275', 8 bytes spill stores, 8 bytes spill loads



79/79 ━━━━━━━━━━━━━━━━━━━━ 227s 887ms/step


In [16]:

label_names = ['melody', 'articulation', 'rhythm_complexity', 'rhythm_stability', 'dissonance', 'atonality', 'mode']
# Generate the classification report
report = classification_report(y_val, y_pred_binary, target_names=label_names)
print(report)

# Generate the multilabel confusion matrix
mcm = multilabel_confusion_matrix(y_val, y_pred_binary)
print("Multilabel Confusion Matrix:")
print(mcm)

                   precision    recall  f1-score   support

           melody       1.00      1.00      1.00       680
     articulation       1.00      1.00      1.00       449
rhythm_complexity       1.00      0.99      1.00       132
 rhythm_stability       1.00      1.00      1.00       769
       dissonance       1.00      1.00      1.00       213
        atonality       1.00      1.00      1.00      1172
             mode       1.00      1.00      1.00       410

        micro avg       1.00      1.00      1.00      3825
        macro avg       1.00      1.00      1.00      3825
     weighted avg       1.00      1.00      1.00      3825
      samples avg       1.00      1.00      1.00      3825

Multilabel Confusion Matrix:
[[[1820    0]
  [   0  680]]

 [[2050    1]
  [   0  449]]

 [[2368    0]
  [   1  131]]

 [[1731    0]
  [   2  767]]

 [[2287    0]
  [   0  213]]

 [[1327    1]
  [   2 1170]]

 [[2089    1]
  [   0  410]]]


In [17]:
import seaborn as sns
import matplotlib.pyplot as plt

# Define label names
label_names = ['melody', 'articulation', 'rhythm_complexity', 'rhythm_stability', 'dissonance', 'atonality', 'mode']

# Plot the confusion matrix for each label
for i, cm in enumerate(mcm):
    print(f'Confusion Matrix for {label_names[i]}')
    print('Actual', cm)
    print("Predicted")
    # plt.figure(figsize=(6, 4))
    # sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted Negative', 'Predicted Positive'], yticklabels=['Actual Negative', 'Actual Positive'])
    # plt.title(f'Confusion Matrix for {label_names[i]}')
    # plt.xlabel('Predicted')
    # plt.ylabel('Actual')
    # plt.show()


Confusion Matrix for melody
Actual [[1820    0]
 [   0  680]]
Predicted
Confusion Matrix for articulation
Actual [[2050    1]
 [   0  449]]
Predicted
Confusion Matrix for rhythm_complexity
Actual [[2368    0]
 [   1  131]]
Predicted
Confusion Matrix for rhythm_stability
Actual [[1731    0]
 [   2  767]]
Predicted
Confusion Matrix for dissonance
Actual [[2287    0]
 [   0  213]]
Predicted
Confusion Matrix for atonality
Actual [[1327    1]
 [   2 1170]]
Predicted
Confusion Matrix for mode
Actual [[2089    1]
 [   0  410]]
Predicted


In [18]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
# import ace_tools as tools
# Define label names
label_names = ['melody', 'articulation', 'rhythm_complexity', 'rhythm_stability', 'dissonance', 'atonality', 'mode']

metrics = []

for i, cm in enumerate(mcm):
    tn, fp = cm[0]
    fn, tp = cm[1]
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (tp + tn) / cm.sum()
    metrics.append({
        'label': label_names[i],
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'accuracy': accuracy
    })

df_metrics = pd.DataFrame(metrics)
# Display metrics table
print(df_metrics)
# tools.display_dataframe_to_user(name="Label-wise Metrics", dataframe=df_metrics)

# # Plot all confusion matrices in a grid
# fig, axes = plt.subplots(2, 3, figsize=(15, 10))
# axes = axes.flatten()
# for i, cm in enumerate(cms):
#     sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
#                 xticklabels=['Pred Neg', 'Pred Pos'],
#                 yticklabels=['True Neg', 'True Pos'],
#                 ax=axes[i])
#     axes[i].set_title(label_names[i])
#     axes[i].set_xlabel('Predicted')
#     axes[i].set_ylabel('Actual')

# plt.tight_layout()
plt.show()

               label  precision    recall  f1_score  accuracy
0             melody   1.000000  1.000000  1.000000    1.0000
1       articulation   0.997778  1.000000  0.998888    0.9996
2  rhythm_complexity   1.000000  0.992424  0.996198    0.9996
3   rhythm_stability   1.000000  0.997399  0.998698    0.9992
4         dissonance   1.000000  1.000000  1.000000    1.0000
5          atonality   0.999146  0.998294  0.998720    0.9988
6               mode   0.997567  1.000000  0.998782    0.9996


In [20]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

thresholds = [0.3, 0.4, 0.5, 0.6]
for t in thresholds:
    preds = (model.predict(x_val) > t).astype(int)
    precision = precision_score(y_val, preds, average='macro')
    recall = recall_score(y_val, preds, average='macro')
    f1 = f1_score(y_val, preds, average='macro')
    accuracy = accuracy_score(y_val, preds)
    print(f"\nThreshold: {t}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1 Score:  {f1:.4f}")
    print(f"  Accuracy:  {accuracy:.4f}")


79/79 ━━━━━━━━━━━━━━━━━━━━ 40s 482ms/step

Threshold: 0.3
  Precision: 0.9991
  Recall:    0.9996
  F1 Score:  0.9993
  Accuracy:  0.9980
79/79 ━━━━━━━━━━━━━━━━━━━━ 43s 521ms/step

Threshold: 0.4
  Precision: 0.9992
  Recall:    0.9996
  F1 Score:  0.9994
  Accuracy:  0.9984
79/79 ━━━━━━━━━━━━━━━━━━━━ 48s 595ms/step

Threshold: 0.5
  Precision: 0.9992
  Recall:    0.9983
  F1 Score:  0.9988
  Accuracy:  0.9976
79/79 ━━━━━━━━━━━━━━━━━━━━ 41s 512ms/step

Threshold: 0.6
  Precision: 0.9992
  Recall:    0.9983
  F1 Score:  0.9988
  Accuracy:  0.9976
